#### 自定义工具

##### **使用[@tool装饰器](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.tool.html#langchain_core.tools.tool)（自定义工具的最简单方式）**

In [1]:
from langchain.tools import tool

@tool
def add_number(a:int,b:int)->int:
    """两个整数相加"""
    return a + b


print(f"name = {add_number.name}")
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")
print(f"return_direct = {add_number.return_direct}")

res = add_number.invoke({"a":10,"b":20})
print(res)

name = add_number
args = {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
description = 两个整数相加
return_direct = False
30


/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


##### 通过@tool的参数设置进行重置

In [2]:
from langchain.tools import tool


@tool(name_or_callable="add_two_number",description="two number add",return_direct=True)
def add_number(a:int,b:int)->int:
    """两个整数相加"""
    return a + b


print(f"name = {add_number.name}")
print(f"description = {add_number.description}")
print(f"args = {add_number.args}")
print(f"return_direct = {add_number.return_direct}")

res = add_number.invoke({"a":10,"b":20})
print(res)

name = add_two_number
description = two number add
args = {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
return_direct = True
30


##### 还可以修改参数的说明

In [3]:
from langchain.tools import tool
from pydantic import BaseModel, Field

class FieldInfo(BaseModel):
    a :int = Field(description="第1个参数")
    b :int = Field(description="第2个参数")

@tool(name_or_callable="add_two_number",description="two number add",args_schema=FieldInfo,return_direct=True)
def add_number(a:int,b:int)->int:
    """两个整数相加"""
    return a + b


print(f"name = {add_number.name}")
print(f"description = {add_number.description}")
print(f"args = {add_number.args}")
print(f"return_direct = {add_number.return_direct}")

res = add_number.invoke({"a":10,"b":20})
print(res)

name = add_two_number
description = two number add
args = {'a': {'description': '第1个参数', 'title': 'A', 'type': 'integer'}, 'b': {'description': '第2个参数', 'title': 'B', 'type': 'integer'}}
return_direct = True
30


#### 方式2：StructuredTool的from_function()

`StructuredTool.from_function` 类方法提供了比`@tool`装饰器更多的可配置性，而无需太多额外的代码。

In [5]:
from langchain_core.tools import StructuredTool

class FieldInfo(BaseModel):
    a :int = Field(description="第1个参数")
    b :int = Field(description="第2个参数")

def add_number(a:int,b:int)->int:
    """两个整数相加"""
    return a + b
tool = add_number_tool = StructuredTool.from_function(
    func=add_number,
    name="add_two_number",
    description="两个整数相加",
    return_direct=True,
    args_schema=FieldInfo
    )

print(f"name = {tool.name}")
print(f"description = {tool.description}")
print(f"args = {tool.args}")
print(f"return_direct = {tool.return_direct}")

res = tool.invoke({"a":10,"b":20})
print(res)

name = add_two_number
description = 两个整数相加
args = {'a': {'description': '第1个参数', 'title': 'A', 'type': 'integer'}, 'b': {'description': '第2个参数', 'title': 'B', 'type': 'integer'}}
return_direct = True
30


In [6]:
#1.导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

# 2.定义LLM模型
model =ChatOpenAI(model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),base_url=os.getenv("OPENAI_BASE_URL"))

# 3.定义工具
tools = [MoveFileTool()]

# 4.将工具转换为函数
functions = [convert_to_openai_function(t) for t in tools]

# print(functions[0])

# 4.模型使用函数
message = model.invoke(
    [HumanMessage(content="move file foo to bar")],
    functions=functions
)

print(message)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 74, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1jwTTPDnwSNmK6xNovj259K978MU', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None} id='run--2c77283d-fd11-4199-999d-4af3caeb9c36-0' usage_metadata={'input_tokens': 74, 'output_tokens': 20, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [7]:
message = model.invoke(
    [HumanMessage(content="how is the weather in beijing")],
    functions=functions
)
print(message)

content="I don't have real-time data access to provide current weather information. However, you can check a reliable weather website or app for the latest weather updates in Beijing." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 76, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1jyBm0boPtFut2sMVm0LlXuqZnTW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--d424964a-5405-464d-bbdf-0c43096c42be-0' usage_metadata={'input_tokens': 76, 'output_tokens': 33, 'total_tokens': 109, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [8]:
message = model.invoke(
    [HumanMessage(content="将本目录下的abc.txt 移动到/Users/zhangyf/PycharmProjects/langchain/model_io")],
    functions=functions
)
print(message)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"./abc.txt","destination_path":"/Users/zhangyf/PycharmProjects/langchain/model_io/abc.txt"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 94, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1k12wuFfl172wj9b7gsoDh1ThVEg', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None} id='run--0c10bb44-360a-4f13-b8a2-a84bed649d18-0' usage_metadata={'input_tokens': 94, 'output_tokens': 38, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [9]:
import json

if "function_call" in message.additional_kwargs:
    tool_name = message.additional_kwargs["function_call"]["name"]
    tool_args = json.loads(message.additional_kwargs["function_call"]["arguments"])
    print(f"调用工具: {tool_name}, 参数: {tool_args}")
else:
    print("模型回复:", message.content)

调用工具: move_file, 参数: {'source_path': './abc.txt', 'destination_path': '/Users/zhangyf/PycharmProjects/langchain/model_io/abc.txt'}


In [10]:
from langchain.tools import MoveFileTool

if "move_file" in message.additional_kwargs["function_call"]["name"]:
    tool = MoveFileTool()
    result = tool.run(tool_args)  # 执行工具
    print("工具执行结果:", result)

工具执行结果: File moved successfully from ./abc.txt to /Users/zhangyf/PycharmProjects/langchain/model_io/abc.txt.
